# DistilBERT fine-tuning으로 감정 분석 모델 학습하기

이번 실습에서는 pre-trained된 DistilBERT를 불러와 이전 주차 실습에서 사용하던 감정 분석 문제에 적용합니다. 먼저 필요한 library들을 불러옵니다.

In [2]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses datasets
# tqdm : 반복문의 진행 상황을 시각적으로 표시하는 progress bar를 제공
# boto3 : Amazon Web Services (AWS) 에 접근하기 위한 SDK
# requests : HTTP 요청을 보내고 응답을 받는 라이브러리
# regex : 정규 표현식을 사용하여 텍스트를 처리하는 라이브러리
# sentencepiece : Google에서 개발한 SentencePiece 모델을 사용하여 텍스트를 subword 단위로 분할하는 라이브러리
# sacremoses : 자연어 처리에서 흔히 사용되는 토큰화, stemming, lemmatization 등의 기능을 제공하는 라이브러리
# datasets : Hugging Face에서 제공하는 Datasets 라이브러리로, 다양한 자연어 처리 데이터셋을 쉽게 다운로드 가능

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00


그 후, 우리가 사용하는 DistilBERT pre-training 때 사용한 tokenizer를 불러옵니다.

In [3]:
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'distilbert-base-uncased')

"""
Hugging Face Transformers 라이브러리에서 DistilBERT 모델의 토크나이저를 로드하는 코드.
토크나이저는 텍스트를 모델이 이해할 수 있는 숫자 시퀀스로 변환하는 역할.

용도:
텍스트 데이터를 DistilBERT 모델에 입력하기 전에 전처리하는 데 사용.
텍스트를 토큰으로 분할하고 각 토큰에 해당하는 ID를 부여.
//
torch.hub.load() 함수:
('huggingface/pytorch-transformers')      : Hugging Face Transformers 라이브러리 저장소.
('tokenizer')                             : 로드할 객체의 종류 (토크나이저).
('distilbert-base-uncased')               : DistilBERT 모델의 uncased 버전 토크나이저를 지정.
distilbert                                : DistilBERT 모델.
base                                      : 기본 크기의 모델.
uncased                                   : 대소문자를 구분하지 않는 모델.
"""

/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://h

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


"\nHugging Face Transformers 라이브러리에서 DistilBERT 모델의 토크나이저를 로드하는 코드.\n토크나이저는 텍스트를 모델이 이해할 수 있는 숫자 시퀀스로 변환하는 역할.\n\n용도:\n텍스트 데이터를 DistilBERT 모델에 입력하기 전에 전처리하는 데 사용.\n텍스트를 토큰으로 분할하고 각 토큰에 해당하는 ID를 부여.\n//\ntorch.hub.load() 함수:\n('huggingface/pytorch-transformers')      : Hugging Face Transformers 라이브러리 저장소.\n('tokenizer')                             : 로드할 객체의 종류 (토크나이저).\n('distilbert-base-uncased')               : DistilBERT 모델의 uncased 버전 토크나이저를 지정.\ndistilbert                                : DistilBERT 모델.\nbase                                      : 기본 크기의 모델.\nuncased                                   : 대소문자를 구분하지 않는 모델.\n"

DistilBERT의 tokenizer를 불러왔으면 이제 `collate_fn`과 data loader를 정의합니다. 이 과정은 이전 실습과 동일하게 다음과 같이 구현할 수 있습니다.

In [4]:
"""
요약:

IMDB 데이터셋을 로드하고, 텍스트 데이터를 DistilBERT 토크나이저로 처리하여
모델 입력에 적합한 형태로 변환하는 collate_fn 함수를 정의하고, 데이터 로더를 생성하는 코드.

용도:

IMDB 데이터셋을 Hugging Face Datasets 라이브러리를 사용하여 로드함.
collate_fn 함수를 정의하여 텍스트 데이터를 일괄 처리하고, 토크나이징, 패딩, 잘라내기를 수행하여 모델 입력에 적합한 텐서 형태로 변환함.
DataLoader를 사용하여 학습 및 테스트 데이터 로더를 생성함.

"""

# ds = load_dataset("stanfordnlp/imdb")
ds = load_dataset("fancyzhx/ag_news")


def collate_fn(batch):
  # del: max_len = 400
  texts, labels = [], []
  for row in batch:
    labels.append(row['label'])
    texts.append(row['text'])

  #texts = torch.LongTensor(tokenizer(texts, padding=True, truncation=True, max_length=max_len).input_ids)
  texts = torch.LongTensor(tokenizer(texts, padding=True).input_ids)
  labels = torch.LongTensor(labels)

  return texts, labels


train_loader = DataLoader(
    ds['train'], batch_size=64, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=64, shuffle=False, collate_fn=collate_fn
)



README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

이제 pre-trained DistilBERT를 불러옵니다. 이번에는 PyTorch hub에서 제공하는 DistilBERT를 불러봅시다.

In [5]:
"""
요약:
Hugging Face Transformers 라이브러리에서 DistilBERT 모델을 로드하는 코드
torch.hub.load() 함수를 사용해서 미리 학습된 DistilBERT 모델을 가져오고 출력.

용도:
텍스트 분류 등 다양한 자연어 처리 작업에 사용할 수 있는 미리 학습된 DistilBERT 모델을 로드.
모델 아키텍처를 확인하고, 추가적인 fine-tuning 또는 수정을 위한 기반으로 사용할 수 있음.
"""
model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
model

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

출력 결과를 통해 우리는 DistilBERT의 architecture는 일반적인 Transformer와 동일한 것을 알 수 있습니다.
Embedding layer로 시작해서 여러 layer의 Attention, FFN를 거칩니다.

---

이제 DistilBERT를 거치고 난 `[CLS]` token의 representation을 가지고 text 분류를 하는 모델을 구현합시다.

In [6]:
from torch import nn

class TextClassifier(nn.Module):
  """
  요약:
  DistilBERT 모델을 사용하여 텍스트 분류를 수행하는 TextClassifier 클래스를 정의하는 코드
  DistilBERT 모델을 인코더로 사용하고, 선형 레이어를 분류기로 사용하여 텍스트를 분류.

  용도:
  DistilBERT를 기반으로 텍스트 분류 모델을 구현함.
  입력 텍스트를 DistilBERT로 인코딩하고, 인코딩된 출력을 선형 레이어에 전달하여 분류 결과를 얻음.
  """
  def __init__(self):
    super().__init__()

    self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased')
    self.classifier = nn.Linear(768, 4) # 출력 차원을 4로 변경

  def forward(self, x):
    x = self.encoder(x)['last_hidden_state']
    x = self.classifier(x[:, 0])

    return x


model = TextClassifier()

"""
self.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased'): DistilBERT 모델을 인코더로 로드. 4번 셀과 동일한 코드임.
self.classifier = nn.Linear(768, 1): 선형 레이어를 분류기로 정의. 입력 크기는 DistilBERT의 출력 크기인 768이고, 출력 크기는 1이면, (이진 분류)임.
def forward(self, x): ...: 모델의 forward pass를 정의. 입력 데이터가 모델을 통과하는 과정을 정의함.
x = self.encoder(x)['last_hidden_state']: 입력 데이터 x를 인코더에 전달하고, 마지막 hidden state를 가져옴.
x = self.classifier(x[:, 0]): 마지막 hidden state의 첫 번째 토큰 ([CLS] 토큰)의 representation을 분류기에 전달.

model = TextClassifier(): TextClassifier 클래스의 인스턴스를 생성하여 model에 할당.

상세 파라미터 설명:

nn.Linear(768, 1):
768: 입력 크기 (DistilBERT 출력 크기).
"""

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


"\nself.encoder = torch.hub.load('huggingface/pytorch-transformers', 'model', 'distilbert-base-uncased'): DistilBERT 모델을 인코더로 로드. 4번 셀과 동일한 코드임.\nself.classifier = nn.Linear(768, 1): 선형 레이어를 분류기로 정의. 입력 크기는 DistilBERT의 출력 크기인 768이고, 출력 크기는 1이면, (이진 분류)임.\ndef forward(self, x): ...: 모델의 forward pass를 정의. 입력 데이터가 모델을 통과하는 과정을 정의함.\nx = self.encoder(x)['last_hidden_state']: 입력 데이터 x를 인코더에 전달하고, 마지막 hidden state를 가져옴.\nx = self.classifier(x[:, 0]): 마지막 hidden state의 첫 번째 토큰 ([CLS] 토큰)의 representation을 분류기에 전달.\n\nmodel = TextClassifier(): TextClassifier 클래스의 인스턴스를 생성하여 model에 할당.\n\n상세 파라미터 설명:\n\nnn.Linear(768, 1):\n768: 입력 크기 (DistilBERT 출력 크기).\n"

위와 같이 `TextClassifier`의 `encoder`를 불러온 DistilBERT, 그리고 `classifier`를 linear layer로 설정합니다.
그리고 `forward` 함수에서 순차적으로 사용하여 예측 결과를 반환합니다.

---

다음은 마지막 classifier layer를 제외한 나머지 부분을 freeze하는 코드를 구현합니다.

In [7]:
"""
요약
DistilBERT 모델의 인코더 부분 가중치를 고정하여 학습되지 않도록 설정.

용도
전이 학습에서 DistilBERT 인코더 부분은 이미 잘 학습되어 있으므로,
새 데이터에 맞게 마지막 분류기 레이어만 학습하기 위해 사용.
"""
for param in model.encoder.parameters():
  param.requires_grad = False

위의 코드는 `encoder`에 해당하는 parameter들의 `requires_grad`를 `False`로 설정하는 모습입니다.
`requires_grad`를 `False`로 두는 경우, gradient 계산 및 업데이트가 이루어지지 않아 결과적으로 학습이 되지 않습니다.
즉, 마지막 `classifier`에 해당하는 linear layer만 학습이 이루어집니다.
이런 식으로 특정 부분들을 freeze하게 되면 효율적으로 학습을 할 수 있습니다.

마지막으로 이전과 같은 코드를 사용하여 학습 결과를 확인해봅시다.

In [9]:
from torch.optim import Adam
import numpy as np
import matplotlib.pyplot as plt


lr = 0.001
model = model.to('cuda')
# loss_fn = nn.BCEWithLogitsLoss()          # 이진 분류를 위한 손실 함수입니다. (주석 처리됨)
loss_fn = nn.CrossEntropyLoss()             # 다중 클래스 분류를 위한 손실 함수를 설정합니다.


optimizer = Adam(model.parameters(), lr=lr) # Adam 옵티마이저를 생성하고, 모델의 파라미터와 학습률을 전달합니다.
n_epochs = 10                               # 학습할 에포크 횟수를 10으로 설정합니다.

for epoch in range(n_epochs):               # 설정된 에포크 횟수만큼 반복합니다.
  total_loss = 0.                           # 에포크마다 총 손실을 0으로 초기화합니다.
  model.train()                             # 모델을 학습 모드로 설정합니다.
  for data in train_loader:                 # train_loader에서 배치 단위로 데이터를 가져옵니다.
    model.zero_grad()                       # 이전 배치의 그래디언트를 초기화합니다.
    inputs, labels = data                   # 배치 데이터를 입력과 레이블로 분리합니다.
    inputs, labels = inputs.to('cuda'), labels.to('cuda').float() # 입력과 레이블을 GPU로 이동시킵니다.

    preds = model(inputs)[..., 0]           # 모델에 입력을 넣어 예측값을 계산합니다.
    loss = loss_fn(preds, labels)           # 예측값과 레이블을 사용하여 손실을 계산합니다.
    loss.backward()                         # 손실을 기반으로 역전파를 수행하여 그래디언트를 계산합니다.
    optimizer.step()                        # 계산된 그래디언트를 사용하여 모델의 가중치를 업데이트합니다.

    total_loss += loss.item()               # 배치 손실을 총 손실에 누적합니다.

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}") # 에포크 번호와 총 손실을 출력합니다.



Epoch   0 | Train Loss: 710266.2707061768
Epoch   1 | Train Loss: 709956.7651367188
Epoch   2 | Train Loss: 709810.7345581055
Epoch   3 | Train Loss: 709795.3737182617
Epoch   4 | Train Loss: 709759.4970550537
Epoch   5 | Train Loss: 709702.6042633057
Epoch   6 | Train Loss: 709766.547668457
Epoch   7 | Train Loss: 709748.1167297363
Epoch   8 | Train Loss: 709731.0691223145
Epoch   9 | Train Loss: 709770.9558563232


In [10]:
def accuracy(model, dataloader):
  cnt = 0
  acc = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    preds = model(inputs)
    preds = torch.argmax(preds, dim=-1)
    preds = (preds > 0).long()[..., 0]

    cnt += labels.shape[0]
    acc += (labels == preds).sum().item()

  return acc / cnt


with torch.no_grad():
  model.eval()
  train_acc = accuracy(model, train_loader)
  test_acc = accuracy(model, test_loader)
  print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")

=========> Train acc: 0.247 | Test acc: 0.256


Loss가 잘 떨어지고, 이전에 우리가 구현한 Transformer보다 더 빨리 수렴하는 것을 알 수 있습니다.